Load Data/Config

In [1]:
"""
generate_annotations_and_crop_and_recognize.py

1) Detect faces in each photo in class folders, generate Pascal VOC XML in ANNOT_DIR.
2) Read XML, crop faces into CROPPED_DIR/<class>.
3) Load TensorFlow/Keras CNN model and class_map, perform real-time face recognition with thresholding.
4) GUI using Tkinter to select paths and trigger processes.

Prerequisites:
    pip install tensorflow opencv-python mtcnn
Place `haarcascade_frontalface_default.xml`, `face_recognition_mobilenet.h5`, and `class_map.pkl` alongside this script.
"""
import os
import cv2
import pickle
import xml.etree.ElementTree as ET
import numpy as np
import threading
import tkinter as tk
from tkinter import filedialog, messagebox
import tensorflow as tf

# Optional MTCNN fallback
try:
    from mtcnn import MTCNN
    mtcnn_detector = MTCNN()
    print("[INFO] MTCNN detector loaded.")
except ImportError:
    mtcnn_detector = None
    print("[WARN] MTCNN not installed; using Haar Cascade only.")

# ----------------------------
# GLOBAL CONFIG DEFAULTS
# ----------------------------
DATASET_DIR = ''
ANNOT_DIR   = ''
CROPPED_DIR = ''
MODEL_PATH  = ''
MAP_PATH    = ''
CASCADE_PATH = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
IMG_SIZE     = (224, 224)
THRESHOLD    = 0.6

# Ensure cascade loaded
face_cascade = cv2.CascadeClassifier(CASCADE_PATH)
if face_cascade.empty():
    raise IOError(f"Failed to load cascade at {CASCADE_PATH}")

# ----------------------------
# FACE DETECTION FUNCTION
# ----------------------------
def detect_faces(img_gray, img_rgb):
    rects = face_cascade.detectMultiScale(img_gray, scaleFactor=1.1, minNeighbors=4)
    faces = rects.tolist() if hasattr(rects, 'tolist') else list(rects)
    if faces:
        return faces
    if mtcnn_detector:
        results = mtcnn_detector.detect_faces(img_rgb)
        return [(r['box'][0], r['box'][1], r['box'][2], r['box'][3]) for r in results]
    return []

# ----------------------------
# GENERATE XML & CROP FACES
# ----------------------------
def generate_and_crop():
    not_detected = []
    if not DATASET_DIR or not ANNOT_DIR or not CROPPED_DIR:
        messagebox.showerror('Error', 'Please set all directory paths!')
        return
    os.makedirs(ANNOT_DIR, exist_ok=True)
    os.makedirs(CROPPED_DIR, exist_ok=True)
    for person in os.listdir(DATASET_DIR):
        person_dir = os.path.join(DATASET_DIR, person)
        if not os.path.isdir(person_dir):
            continue
        output_folder = os.path.join(CROPPED_DIR, person)
        os.makedirs(output_folder, exist_ok=True)
        for img_name in os.listdir(person_dir):
            if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
            img_path = os.path.join(person_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            rgb  = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            faces = detect_faces(gray, rgb)
            if not faces:
                not_detected.append(img_path)
            # Build XML
            ann = ET.Element('annotation')
            ET.SubElement(ann, 'folder').text   = person
            ET.SubElement(ann, 'filename').text = img_name
            h, w = img.shape[:2]
            size = ET.SubElement(ann, 'size')
            ET.SubElement(size, 'width').text  = str(w)
            ET.SubElement(size, 'height').text = str(h)
            ET.SubElement(size, 'depth').text  = str(img.shape[2])
            for (x, y, fw, fh) in faces:
                obj = ET.SubElement(ann, 'object')
                ET.SubElement(obj, 'name').text = 'face'
                bnd = ET.SubElement(obj, 'bndbox')
                ET.SubElement(bnd, 'xmin').text = str(max(0, x))
                ET.SubElement(bnd, 'ymin').text = str(max(0, y))
                ET.SubElement(bnd, 'xmax').text = str(min(w, x+fw))
                ET.SubElement(bnd, 'ymax').text = str(min(h, y+fh))
            xml_path = os.path.join(ANNOT_DIR, os.path.splitext(img_name)[0] + '.xml')
            ET.ElementTree(ann).write(xml_path, encoding='utf-8', xml_declaration=True)
            # Crop faces
            for idx, (x, y, fw, fh) in enumerate(faces, start=1):
                crop = img[y:y+fh, x:x+fw]
                if crop.size == 0:
                    continue
                out_name = f"{os.path.splitext(img_name)[0]}_face{idx}.jpg"
                cv2.imwrite(os.path.join(output_folder, out_name), crop)
    if not_detected:
        messagebox.showwarning('Warning', f"{len(not_detected)} images had no faces detected.")

# ----------------------------
# REAL-TIME FACE RECOGNITION
# ----------------------------
def recognize_live():
    if not MODEL_PATH or not MAP_PATH:
        messagebox.showerror('Error', 'Please set model and map files!')
        return
    model = tf.keras.models.load_model(MODEL_PATH)
    with open(MAP_PATH, 'rb') as f:
        class_map = pickle.load(f)
    cap = cv2.VideoCapture(0)
    print("[*] Press ESC to exit")
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        dets = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
        for (x, y, w, h) in dets:
            face = frame[y:y+h, x:x+w]
            face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face_resized = cv2.resize(face_rgb, IMG_SIZE)
            face_norm = face_resized.astype('float32') / 255.0
            preds = model.predict(np.expand_dims(face_norm, axis=0))[0]
            idx = int(np.argmax(preds)); prob = float(preds[idx])
            if prob < THRESHOLD:
                lbl, col = 'Unknown', (0, 0, 255)
            else:
                lbl, col = class_map[idx], (0, 255, 0)
            text = lbl if lbl=='Unknown' else f"{lbl} ({prob*100:.1f}%)"
            cv2.rectangle(frame, (x, y), (x+w, y+h), col, 2)
            cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, col, 2)
        cv2.imshow('Face Recognition', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

# ----------------------------
# GUI (Tkinter)
# ----------------------------
root = tk.Tk()
root.title('Face Recognition Tool')

# Function to create path selection rows

def make_path_row(label, var_name, row, file=False, types=None):
    tk.Label(root, text=label).grid(row=row, column=0, sticky='w')
    entry = tk.Entry(root, width=40)
    entry.grid(row=row, column=1)
    def browse():
        path = filedialog.askopenfilename(filetypes=types) if file else filedialog.askdirectory()
        if path:
            globals()[var_name] = path
            entry.delete(0, tk.END)
            entry.insert(0, path)
    btn = tk.Button(root, text='Browse', command=browse)
    btn.grid(row=row, column=2)
    return entry

# Generate GUI rows
ent_ds = make_path_row('Dataset Dir:', 'DATASET_DIR', 0)
ent_an = make_path_row('Annotations Dir:', 'ANNOT_DIR', 1)
ent_cr = make_path_row('Cropped Dir:', 'CROPPED_DIR', 2)
ent_md = make_path_row('Model (.h5):', 'MODEL_PATH', 3, True, [('H5 files','*.h5')])
ent_mp = make_path_row('Map (.pkl):', 'MAP_PATH', 4, True, [('PKL files','*.pkl')])

# Action Buttons
btn_gen = tk.Button(root, text='Generate & Crop', width=20,
                    command=lambda: threading.Thread(target=generate_and_crop).start())
btn_gen.grid(row=5, column=1, pady=5)

btn_rec = tk.Button(root, text='Start Recognition', width=20,
                    command=lambda: threading.Thread(target=recognize_live).start())
btn_rec.grid(row=6, column=1, pady=5)

root.mainloop()


ModuleNotFoundError: No module named 'cv2'

In [ ]:
import os
import cv2
import pickle
import numpy as np
import threading
import tkinter as tk
from tkinter import filedialog, messagebox
import tensorflow as tf
from datetime import datetime

try:
    from mtcnn import MTCNN
    mtcnn_detector = MTCNN()
    print("[INFO] MTCNN detector loaded.")
except ImportError:
    mtcnn_detector = None
    print("[WARN] MTCNN not installed; only Haar Cascade will be used.")

# GLOBAL CONFIG
DATASET_DIR = ''
ANNOT_DIR   = ''
CROPPED_DIR = ''
MODEL_PATH  = ''
MAP_PATH    = ''
CASCADE_PATH = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
IMG_SIZE     = (224, 224)
THRESHOLD    = 0.6
ATTENDANCE_FILE = 'attendance.csv'

face_cascade = cv2.CascadeClassifier(CASCADE_PATH)
if face_cascade.empty():
    raise IOError(f"Failed to load cascade from {CASCADE_PATH}")

def detect_faces(img_gray, img_rgb):
    rects = face_cascade.detectMultiScale(img_gray, scaleFactor=1.1, minNeighbors=4)
    faces = rects.tolist() if hasattr(rects, 'tolist') else list(rects)
    if faces:
        return faces
    if mtcnn_detector:
        results = mtcnn_detector.detect_faces(img_rgb)
        return [(r['box'][0], r['box'][1], r['box'][2], r['box'][3]) for r in results]
    return []

def recognize_live_attendance():
    if not MODEL_PATH or not MAP_PATH:
        messagebox.showerror('Error', 'Please set model (.h5) and map (.pkl) files first!')
        return

    model = tf.keras.models.load_model(MODEL_PATH)
    with open(MAP_PATH, 'rb') as f:
        class_map = pickle.load(f)

    os.makedirs(os.path.dirname(ATTENDANCE_FILE) or '.', exist_ok=True)
    had_attendance = set()
    if not os.path.isfile(ATTENDANCE_FILE):
        with open(ATTENDANCE_FILE, 'w') as f:
            f.write("Name,Timestamp\n")

    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    if not cap.isOpened():
        messagebox.showerror('Error', 'Cannot open webcam!')
        return

    print("[*] Press ESC to exit.")
    frame_count = 0
    SKIP_FRAMES = 3

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        orig_h, orig_w = frame.shape[:2]

        # Resize frame untuk deteksi lebih cepat
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        gray_small = cv2.cvtColor(small_frame, cv2.COLOR_BGR2GRAY)
        rgb_small = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        faces_small = detect_faces(gray_small, rgb_small)

        # Gambar kotak di frame asli (warna abu-abu)
        for (x_s, y_s, w_s, h_s) in faces_small:
            x = int(x_s * 2)
            y = int(y_s * 2)
            w = int(w_s * 2)
            h = int(h_s * 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (200, 200, 200), 1)

        # Recognition tiap SKIP_FRAMES frame
        if frame_count % SKIP_FRAMES == 0 and faces_small:
            for (x_s, y_s, w_s, h_s) in faces_small:
                x = int(x_s * 2)
                y = int(y_s * 2)
                w = int(w_s * 2)
                h = int(h_s * 2)

                face = frame[y:y+h, x:x+w]
                if face.size == 0:
                    continue
                face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face_resized = cv2.resize(face_rgb, IMG_SIZE)
                face_norm = face_resized.astype('float32') / 255.0
                preds = model.predict(np.expand_dims(face_norm, axis=0), verbose=0)[0]
                idx = int(np.argmax(preds))
                prob = float(preds[idx])

                if prob < THRESHOLD:
                    lbl, col = 'Unknown', (0, 0, 255)
                else:
                    lbl, col = class_map[idx], (0, 255, 0)
                    if lbl not in had_attendance:
                        had_attendance.add(lbl)
                        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        with open(ATTENDANCE_FILE, 'a') as f:
                            f.write(f"{lbl},{timestamp}\n")

                text = lbl if lbl == 'Unknown' else f"{lbl} ({prob*100:.1f}%)"
                cv2.rectangle(frame, (x, y), (x+w, y+h), col, 2)
                cv2.putText(frame, text, (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, col, 2)

        frame_count += 1
        cv2.imshow('Attendance - Face Recognition', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo('Finished', f"Attendance saved to '{ATTENDANCE_FILE}'.")

# GUI Tkinter
root = tk.Tk()
root.title('Face Recognition Attendance Tool')

def make_path_row(label, var_name, row, file=False, types=None):
    tk.Label(root, text=label).grid(row=row, column=0, sticky='w', padx=5, pady=3)
    entry = tk.Entry(root, width=40)
    entry.grid(row=row, column=1, padx=5, pady=3)
    def browse():
        path = filedialog.askopenfilename(filetypes=types) if file else filedialog.askdirectory()
        if path:
            globals()[var_name] = path
            entry.delete(0, tk.END)
            entry.insert(0, path)
    btn = tk.Button(root, text='Browse', command=browse)
    btn.grid(row=row, column=2, padx=5, pady=3)
    return entry

ent_ds = make_path_row('Dataset Dir:',    'DATASET_DIR', 0)
ent_an = make_path_row('Annotations Dir:', 'ANNOT_DIR',   1)
ent_cr = make_path_row('Cropped Dir:',     'CROPPED_DIR', 2)
ent_md = make_path_row('Model (.h5):',     'MODEL_PATH',  3, True, [('H5 files','*.h5')])
ent_mp = make_path_row('Map (.pkl):',      'MAP_PATH',    4, True, [('PKL files','*.pkl')])

btn_gen = tk.Button(root, text='Generate & Crop', width=20,
                    command=lambda: threading.Thread(target=lambda: messagebox.showinfo('Info', 'Use your own generate_and_crop function')).start())
btn_gen.grid(row=5, column=1, pady=10)

btn_rec = tk.Button(root, text='Start Attendance', width=20,
                    command=lambda: threading.Thread(target=recognize_live_attendance).start())
btn_rec.grid(row=6, column=1, pady=10)

root.mainloop()


: 

In [4]:
import os
import pickle

CROPPED_DIR = 'D:/Python/FR Mulmed/dataset_cropped'  
# Misalnya: "/home/user/dataset/cropped_faces"

def build_class_map(cropped_dir, output_map_path):
    # Ambil nama‐nama subfolder (kelas) dan urutkan
    class_names = sorted([d for d in os.listdir(cropped_dir)
                          if os.path.isdir(os.path.join(cropped_dir, d))])
    # Buat mapping nama→index
    class_map = {name: idx for idx, name in enumerate(class_names)}

    # Simpan ke pickle
    with open(output_map_path, 'wb') as f:
        pickle.dump(class_map, f)
    print(f"[INFO] class_map.pkl dibuat: {output_map_path}")
    print(class_map)

if __name__ == '__main__':
    out_path = 'class_map.pkl'
    build_class_map(CROPPED_DIR, out_path)


[INFO] class_map.pkl dibuat: class_map.pkl
{'Daffa Ananta Rachman': 0, 'Farhan Muamar Fawwaz': 1, 'Hellen Allysa Putri': 2, 'MUHAMMAD BINTANG PRAJUDHA': 3, 'Muhammad Adlan Hafizha': 4, 'Muhammad Nabil Alfarizi': 5, 'Nesya Sulistyawati': 6, 'REVAN AHMAD ZAYYAN': 7, 'Raditya Darma Sakti': 8, 'Rizka Ananda Pratama': 9}


In [9]:
"""
train_face_model.py

- Menggunakan data pada CROPPED_DIR yang terstruktur per kelas:
    CROPPED_DIR/
    ├── Alice/
    │   ├── alice1.jpg
    │   ├── alice2.jpg
    │   └── ...
    ├── Bob/
    │   └── bob1.jpg ...
    └── Carol/
        └── carol1.jpg ...
- Resize semua gambar menjadi 224×224 (sesuai IMG_SIZE).
- Fine‐tune MobileNetV2 sebagai feature extractor, tambahkan classification head.
- Simpan model final ke .h5 dan class_map.pkl.
"""

import os
import pickle
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Lokasi dataset hasil crop
CROPPED_DIR = 'D:/Python/FR Mulmed/dataset_cropped'  
# Direktori output model & map
OUTPUT_MODEL = 'face_recognition_mobilenet.h5'
OUTPUT_MAP   = 'class_map.pkl'

# Parameter
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-4
VALIDATION_SPLIT = 0.2

# 1) Bangun class_map terlebih dahulu
def build_class_map(cropped_dir, output_map_path):
    class_names = sorted([d for d in os.listdir(cropped_dir)
                          if os.path.isdir(os.path.join(cropped_dir, d))])
    class_map = {name: idx for idx, name in enumerate(class_names)}
    with open(output_map_path, 'wb') as f:
        pickle.dump(class_map, f)
    print("[INFO] class_map dibuat:", class_map)
    return class_map

# 2) Persiapkan ImageDataGenerator untuk training & validation
def create_generators(cropped_dir, img_size, batch_size, val_split):
    # Data augmentation untuk training
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        validation_split=val_split
    )

    # Generator untuk training
    train_gen = train_datagen.flow_from_directory(
        cropped_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )
    # Generator untuk validation
    valid_gen = train_datagen.flow_from_directory(
        cropped_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )
    return train_gen, valid_gen

# 3) Bangun model (MobileNetV2 + custom head)
def build_model(num_classes, img_size, learning_rate):
    base = MobileNetV2(include_top=False, weights='imagenet',
                       input_shape=(img_size[0], img_size[1], 3))
    base.trainable = False  # freeze semua layer base

    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


if __name__ == '__main__':
    # 1) Buat class_map dan simpan
    class_map = build_class_map(CROPPED_DIR, OUTPUT_MAP)
    num_classes = len(class_map)

    # 2) Siapkan generators
    train_gen, valid_gen = create_generators(CROPPED_DIR, IMG_SIZE,
                                             BATCH_SIZE, VALIDATION_SPLIT)

    # 3) Bangun model
    model = build_model(num_classes, IMG_SIZE, LEARNING_RATE)
    model.summary()

    # 4) Latih model
    history = model.fit(
        train_gen,
        epochs=EPOCHS,
        validation_data=valid_gen
    )

    # 5) (Opsional) Unfreeze base dan fine‐tune sebagian layer atas
    # base.trainable = True
    # for layer in base.layers[:-20]:
    #     layer.trainable = False
    # model.compile(optimizer=Adam(lr=LEARNING_RATE/10),
    #               loss='categorical_crossentropy',
    #               metrics=['accuracy'])
    # history_finetune = model.fit(
    #     train_gen,
    #     epochs=5,
    #     validation_data=valid_gen
    # )

    # 6) Simpan model ke .h5
    model.save(OUTPUT_MODEL)
    print(f"[INFO] Model disimpan ke {OUTPUT_MODEL}")


[INFO] class_map dibuat: {'Daffa Ananta Rachman': 0, 'Farhan Muamar Fawwaz': 1, 'Hellen Allysa Putri': 2, 'MUHAMMAD BINTANG PRAJUDHA': 3, 'Muhammad Adlan Hafizha': 4, 'Muhammad Nabil Alfarizi': 5, 'Nesya Sulistyawati': 6, 'REVAN AHMAD ZAYYAN': 7, 'Raditya Darma Sakti': 8, 'Rizka Ananda Pratama': 9}
Found 109 images belonging to 10 classes.
Found 20 images belonging to 10 classes.


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.0962 - loss: 2.9584 - val_accuracy: 0.0000e+00 - val_loss: 2.9530
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 911ms/step - accuracy: 0.0778 - loss: 2.8890 - val_accuracy: 0.0000e+00 - val_loss: 2.9351
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 992ms/step - accuracy: 0.0647 - loss: 2.9596 - val_accuracy: 0.0000e+00 - val_loss: 2.7939
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.0821 - loss: 2.8067 - val_accuracy: 0.1000 - val_loss: 2.6974
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.0684 - loss: 2.8185 - val_accuracy: 0.0000e+00 - val_loss: 2.6881
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.1773 - loss: 2.8928 - val_accuracy: 0.0500 - val_loss: 2.6549
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 908ms/step - accuracy: 0.0606 - loss: 2.7505 - val_accuracy: 0.0500 - val_loss: 2.5101
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 950ms/step - accuracy: 0.1935 - loss: 2.6086 - val_accuracy: 0.1500 - val_

[INFO] Model disimpan ke face_recognition_mobilenet.h5


In [1]:
#!/usr/bin/env python3
# ===============================================
# Face-Recognition Attendance   (LOCAL – Windows)
# ===============================================

import os, csv, cv2, time, numpy as np, face_recognition
from datetime import datetime

# ───────────── Konfigurasi ─────────────
DATASET_PATH = r"D:\Python\FR Mulmed"   # <- ganti jika perlu
CSV_PATH     = "Attendance.csv"         # akan dibuat di folder skrip
CAMERA_INDEX = 0                        # 0 = webcam default
FRAME_SCALE  = 0.25                     # downscale utk kecepatan

# ────────── Muat encodings ────────────
def load_dataset(path):
    encs, names = [], []
    if not os.path.isdir(path):
        raise FileNotFoundError(f"Folder dataset tidak ditemukan: {path}")
    for person in os.listdir(path):
        folder = os.path.join(path, person)
        if not os.path.isdir(folder): continue
        for file in os.listdir(folder):
            if os.path.splitext(file)[1].lower() not in (".jpg",".jpeg",".png"):
                continue
            img = cv2.imread(os.path.join(folder, file))
            if img is None: continue
            e = face_recognition.face_encodings(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            if e: encs.append(e[0]); names.append(person)
    print(f"[INFO] Loaded {len(encs)} face encodings.")
    return encs, names

print("=== Scanning dataset … ===")
encodeListKnown, studentNames = load_dataset(DATASET_PATH)
print("=== Dataset loaded ===\n")

# ───────── Attendance helper ──────────
def mark_attendance(name):
    today = datetime.now().strftime("%Y-%m-%d")
    header_needed = not os.path.exists(CSV_PATH)
    with open(CSV_PATH, "a+", newline="") as f:
        f.seek(0)
        if header_needed:
            csv.writer(f).writerow(["Name","Date","Time"])
        for row in csv.reader(f):
            if row and row[0]==name and row[1]==today:
                return False
        csv.writer(f).writerow([name, today, datetime.now().strftime("%H:%M:%S")])
    return True

# ────────── Jalankan webcam ───────────
print("[INFO] Tekan  q  untuk keluar.")
cap = cv2.VideoCapture(CAMERA_INDEX)
if not cap.isOpened():
    raise RuntimeError("Webcam tidak bisa dibuka.")

while True:
    ret, frame = cap.read()
    if not ret: break
    small = cv2.resize(frame, (0,0), fx=FRAME_SCALE, fy=FRAME_SCALE)
    rgb   = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)

    locs = face_recognition.face_locations(rgb)
    encs = face_recognition.face_encodings(rgb, locs)

    for loc, enc in zip(locs, encs):
        top,right,bottom,left = [int(v/FRAME_SCALE) for v in (*loc,)]
        matches  = face_recognition.compare_faces(encodeListKnown, enc)
        dists    = face_recognition.face_distance(encodeListKnown, enc)
        idx      = np.argmin(dists) if len(dists) else None

        name, color, label = "Unknown", (0,0,255), "Unknown"
        if idx is not None and matches[idx]:
            name = studentNames[idx]
            if mark_attendance(name):
                label, color = f"{name} ✔", (0,255,0)
            else:
                label, color = f"{name} (tercatat)", (0,255,0)
        cv2.rectangle(frame,(left,top),(right,bottom),color,2)
        cv2.rectangle(frame,(left,bottom-25),(right,bottom),color,cv2.FILLED)
        cv2.putText(frame,label,(left+6,bottom-6),cv2.FONT_HERSHEY_SIMPLEX,0.55,(255,255,255),1)

    cv2.imshow("Face Attendance", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release(); cv2.destroyAllWindows()
print("[INFO] Program berakhir.")


ModuleNotFoundError: No module named 'face_recognition'

In [ ]:
import os
import cv2
import pickle
import numpy as np
import threading
import tkinter as tk
from tkinter import filedialog, messagebox
import tensorflow as tf
from datetime import datetime

try:
    from mtcnn import MTCNN
    mtcnn_detector = MTCNN()
    print("[INFO] MTCNN detector loaded.")
except ImportError:
    mtcnn_detector = None
    print("[WARN] MTCNN not installed; only Haar Cascade will be used.")

# GLOBAL CONFIG
DATASET_DIR = ''
ANNOT_DIR   = ''
CROPPED_DIR = ''
MODEL_PATH  = ''
MAP_PATH    = ''
CASCADE_PATH = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
IMG_SIZE     = (224, 224)
THRESHOLD    = 0.6
ATTENDANCE_FILE = 'attendance.csv'

face_cascade = cv2.CascadeClassifier(CASCADE_PATH)
if face_cascade.empty():
    raise IOError(f"Failed to load cascade from {CASCADE_PATH}")

def detect_faces(img_gray, img_rgb):
    rects = face_cascade.detectMultiScale(img_gray, scaleFactor=1.1, minNeighbors=4)
    faces = rects.tolist() if hasattr(rects, 'tolist') else list(rects)
    if faces:
        return faces
    if mtcnn_detector:
        results = mtcnn_detector.detect_faces(img_rgb)
        return [(r['box'][0], r['box'][1], r['box'][2], r['box'][3]) for r in results]
    return []

def recognize_live_attendance():
    if not MODEL_PATH or not MAP_PATH:
        messagebox.showerror('Error', 'Please set model (.h5) and map (.pkl) files first!')
        return

    model = tf.keras.models.load_model(MODEL_PATH)
    with open(MAP_PATH, 'rb') as f:
        class_map = pickle.load(f)

    os.makedirs(os.path.dirname(ATTENDANCE_FILE) or '.', exist_ok=True)
    had_attendance = set()
    if not os.path.isfile(ATTENDANCE_FILE):
        with open(ATTENDANCE_FILE, 'w') as f:
            f.write("Name,Timestamp\n")

    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    if not cap.isOpened():
        messagebox.showerror('Error', 'Cannot open webcam!')
        return

    print("[*] Press ESC to exit.")
    frame_count = 0
    SKIP_FRAMES = 3

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        orig_h, orig_w = frame.shape[:2]

        # Resize frame untuk deteksi lebih cepat
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        gray_small = cv2.cvtColor(small_frame, cv2.COLOR_BGR2GRAY)
        rgb_small = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        faces_small = detect_faces(gray_small, rgb_small)

        # Gambar kotak di frame asli (warna abu-abu)
        for (x_s, y_s, w_s, h_s) in faces_small:
            x = int(x_s * 2)
            y = int(y_s * 2)
            w = int(w_s * 2)
            h = int(h_s * 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (200, 200, 200), 1)

        # Recognition tiap SKIP_FRAMES frame
        if frame_count % SKIP_FRAMES == 0 and faces_small:
            for (x_s, y_s, w_s, h_s) in faces_small:
                x = int(x_s * 2)
                y = int(y_s * 2)
                w = int(w_s * 2)
                h = int(h_s * 2)

                face = frame[y:y+h, x:x+w]
                if face.size == 0:
                    continue
                face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face_resized = cv2.resize(face_rgb, IMG_SIZE)
                face_norm = face_resized.astype('float32') / 255.0
                preds = model.predict(np.expand_dims(face_norm, axis=0), verbose=0)[0]
                idx = int(np.argmax(preds))
                prob = float(preds[idx])

                if prob < THRESHOLD:
                    lbl, col = 'Unknown', (0, 0, 255)
                else:
                    lbl, col = class_map[idx], (0, 255, 0)
                    if lbl not in had_attendance:
                        had_attendance.add(lbl)
                        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        with open(ATTENDANCE_FILE, 'a') as f:
                            f.write(f"{lbl},{timestamp}\n")

                text = lbl if lbl == 'Unknown' else f"{lbl} ({prob*100:.1f}%)"
                cv2.rectangle(frame, (x, y), (x+w, y+h), col, 2)
                cv2.putText(frame, text, (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, col, 2)

        frame_count += 1
        cv2.imshow('Attendance - Face Recognition', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo('Finished', f"Attendance saved to '{ATTENDANCE_FILE}'.")

# GUI Tkinter
root = tk.Tk()
root.title('Face Recognition Attendance Tool')

def make_path_row(label, var_name, row, file=False, types=None):
    tk.Label(root, text=label).grid(row=row, column=0, sticky='w', padx=5, pady=3)
    entry = tk.Entry(root, width=40)
    entry.grid(row=row, column=1, padx=5, pady=3)
    def browse():
        path = filedialog.askopenfilename(filetypes=types) if file else filedialog.askdirectory()
        if path:
            globals()[var_name] = path
            entry.delete(0, tk.END)
            entry.insert(0, path)
    btn = tk.Button(root, text='Browse', command=browse)
    btn.grid(row=row, column=2, padx=5, pady=3)
    return entry

ent_ds = make_path_row('Dataset Dir:',    'DATASET_DIR', 0)
ent_an = make_path_row('Annotations Dir:', 'ANNOT_DIR',   1)
ent_cr = make_path_row('Cropped Dir:',     'CROPPED_DIR', 2)
ent_md = make_path_row('Model (.h5):',     'MODEL_PATH',  3, True, [('H5 files','*.h5')])
ent_mp = make_path_row('Map (.pkl):',      'MAP_PATH',    4, True, [('PKL files','*.pkl')])

btn_gen = tk.Button(root, text='Generate & Crop', width=20,
                    command=lambda: threading.Thread(target=lambda: messagebox.showinfo('Info', 'Implement generate_and_crop separately')).start())
btn_gen.grid(row=5, column=1, pady=10)

btn_rec = tk.Button(root, text='Start Attendance', width=20,
                    command=lambda: threading.Thread(target=recognize_live_attendance).start())
btn_rec.grid(row=6, column=1, pady=10)

root.mainloop()


[INFO] MTCNN detector loaded.


[*] Press ESC to exit.


In [12]:
"""
Face-Recognition Attendance – Anti-Foto + Notifikasi
---------------------------------------------------
• Banner merah “ANOMALI!” → wajah unknown/spoof
• Banner hijau “SUDAH ABSEN” → presensi berhasil
• Liveness = move ≥10 px AND (blink OR pose≥10°) AND flowRatio>1.3
"""

from __future__ import annotations
import time, pickle
from datetime import datetime
from pathlib import Path
from typing import Dict, Tuple, Any, List

import cv2, face_recognition, numpy as np


# ═════ konfigurasi ═════
DATASET_PATH, ENCODE_FILE, ATTENDANCE_FILE = (
    Path("dataset"), Path("EncodeFile.p"), Path("attendance.csv")
)
CAMERA_INDEX           = 0
SCALE                  = 0.25
EVERY_N                = 2
ANOMALY_TIME           = 2.0
SUCCESS_TIME           = 2.0          # banner hijau

MOVE_PX, WIN_SEC       = 10, 3.0
EAR_THR, EAR_SEQ, BLINK_REQ = 0.21, 2, 1
POSE_DEG               = 10
FLOW_RATIO_THR         = 1.3
# ══════════════════════


# ── helper visual ────────────────────────────────────────────────────────
def draw_banner(img, text, color):
    h, w = img.shape[:2]; bar = 60
    ov = img.copy(); cv2.rectangle(ov, (0,0), (w,bar), color, -1)
    img[:] = cv2.addWeighted(ov, .6, img, .4, 0)
    cv2.putText(img, text, (10, int(bar*0.7)), cv2.FONT_HERSHEY_SIMPLEX,
                1.5, (255,255,255), 3, cv2.LINE_AA)

def warn_banner(img):  draw_banner(img, "⚠  ANOMALI!  ⚠", (0,0,255))
def ok_banner(img):    draw_banner(img, "✓  SUDAH ABSEN", (0,180,0))
# ─────────────────────────────────────────────────────────────────────────


def ear(eye):  # eye = list[(x,y)]
    A = np.linalg.norm(np.subtract(eye[1], eye[5]))
    B = np.linalg.norm(np.subtract(eye[2], eye[4]))
    C = np.linalg.norm(np.subtract(eye[0], eye[3]))
    return (A+B)/(2.0*C)


MODEL_3D = np.array([
    (0,0,0),(0,-330,-65),(-225,170,-135),(225,170,-135),
    (-150,-150,-125),(150,-150,-125)
], dtype="double")

def pose_pitch_yaw(lm, shape):
    pts2d = np.float32([
        lm['nose_tip'][0], lm['chin'][0],
        lm['left_eye_corner'][0], lm['right_eye_corner'][0],
        lm['mouth_left'][0], lm['mouth_right'][0]])
    f = shape[1]
    cam = np.array([[f,0,shape[1]/2],[0,f,shape[0]/2],[0,0,1]], dtype="double")
    ok,rvec,tvec = cv2.solvePnP(MODEL_3D, pts2d, cam, np.zeros((4,1)),
                                flags=cv2.SOLVEPNP_ITERATIVE)
    if not ok: return 0.0,0.0
    rmat,_ = cv2.Rodrigues(rvec); proj = cv2.hconcat([rmat,tvec])
    *_,euler = cv2.decomposeProjectionMatrix(proj)
    return float(euler[0]), float(euler[1])        # pitch, yaw


def flow_ratio(prev, curr, box):
    l,t,r,b = box
    prev_g = cv2.cvtColor(prev[t:b,l:r], cv2.COLOR_BGR2GRAY)
    curr_g = cv2.cvtColor(curr[t:b,l:r], cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_g,curr_g,None,0.5,3,15,3,5,1.2,0)
    mag = np.linalg.norm(flow,axis=2)
    h,w = mag.shape; y0,y1 = int(.25*h), int(.75*h); x0,x1 = int(.25*w), int(.75*w)
    inner = mag[y0:y1,x0:x1]
    edge  = mag.copy(); edge[y0:y1,x0:x1]=0
    return (edge.mean()+1e-6)/(inner.mean()+1e-6)


# ── encode helpers ───────────────────────────────────────────────────────
def gen_enc():
    imgs,nms=[],[]
    for p in DATASET_PATH.iterdir():
        if p.is_dir():
            for f in p.glob("*"):
                im=cv2.imread(str(f))
                if im is not None:
                    imgs.append(im); nms.append(p.name)
    if not imgs: raise RuntimeError("Dataset kosong.")
    enc=[face_recognition.face_encodings(cv2.cvtColor(i,cv2.COLOR_BGR2RGB))[0]
         for i in imgs if face_recognition.face_encodings(cv2.cvtColor(i,cv2.COLOR_BGR2RGB))]
    ENCODE_FILE.write_bytes(pickle.dumps((enc,nms)))

def load_enc():
    if not ENCODE_FILE.exists(): gen_enc()
    return pickle.loads(ENCODE_FILE.read_bytes())


def mark(name:str)->bool:
    today=datetime.now().strftime("%Y-%m-%d")
    if not ATTENDANCE_FILE.exists():
        ATTENDANCE_FILE.write_text("Name,Time\n")
    rows=ATTENDANCE_FILE.read_text().splitlines()[1:]
    if any(r.startswith(f"{name},{today}") for r in rows):
        return False
    ATTENDANCE_FILE.open("a").write(f"{name},{datetime.now():%Y-%m-%d %H:%M:%S}\n")
    print(f"[ATTENDANCE] {name}")
    return True
# ─────────────────────────────────────────────────────────


def main():
    encs,nms = load_enc()
    cap = cv2.VideoCapture(CAMERA_INDEX, cv2.CAP_DSHOW)
    if not cap.isOpened(): raise RuntimeError("Webcam gagal dibuka.")
    cv2.namedWindow("Attendance", cv2.WINDOW_NORMAL)

    cache:Dict[str,Dict[str,Any]]={}
    prev=None; i=0
    warn_until = ok_until = 0.0

    while True:
        ret,frame=cap.read();  now=time.time()
        if not ret: break
        small=cv2.resize(frame,(0,0),fx=SCALE,fy=SCALE)
        rgb=cv2.cvtColor(small,cv2.COLOR_BGR2RGB)

        if i%EVERY_N==0:
            locs=face_recognition.face_locations(rgb)
            encf=face_recognition.face_encodings(rgb,locs)

            for enc,loc in zip(encf,locs):
                dist=face_recognition.face_distance(encs,enc)
                idx=int(np.argmin(dist))
                match=dist[idx]<0.48
                name=nms[idx].upper() if match else "UNKNOWN"

                t,r,b,l=[int(c/SCALE) for c in loc]
                center=((l+r)//2,(t+b)//2)

                lm=face_recognition.face_landmarks(rgb,[loc])[0]
                le=[(int(x/SCALE),int(y/SCALE)) for x,y in lm["left_eye"]]
                re=[(int(x/SCALE),int(y/SCALE)) for x,y in lm["right_eye"]]
                ear_val=(ear(le)+ear(re))/2

                lm_pose={
                    'nose_tip':[(int(lm["nose_tip"][2][0]/SCALE),int(lm["nose_tip"][2][1]/SCALE))],
                    'chin':[(int(lm["chin"][8][0]/SCALE),int(lm["chin"][8][1]/SCALE))],
                    'left_eye_corner':[(int(lm["left_eye"][0][0]/SCALE),int(lm["left_eye"][0][1]/SCALE))],
                    'right_eye_corner':[(int(lm["right_eye"][3][0]/SCALE),int(lm["right_eye"][3][1]/SCALE))],
                    'mouth_left':[(int(lm["top_lip"][0][0]/SCALE),int(lm["top_lip"][0][1]/SCALE))],
                    'mouth_right':[(int(lm["top_lip"][6][0]/SCALE),int(lm["top_lip"][6][1]/SCALE))]
                }
                pitch,yaw = pose_pitch_yaw(lm_pose,frame.shape[:2])

                flow_ok=True
                if prev is not None:
                    flow_ok = flow_ratio(prev,frame,(l,t,r,b)) > FLOW_RATIO_THR

                c=cache.setdefault(name,{"center":center,"move_t":now,
                                         "ear_ctr":0,"blinks":0,
                                         "pose":(pitch,yaw),"pose_t":now,
                                         "flow_ok":flow_ok})

                # update blink ctr
                if ear_val<EAR_THR: c["ear_ctr"]+=1
                else:
                    if c["ear_ctr"]>=EAR_SEQ: c["blinks"]+=1
                    c["ear_ctr"]=0

                # move
                if np.linalg.norm(np.subtract(center,c["center"]))>MOVE_PX:
                    c["center"],c["move_t"]=center,now

                # pose
                if abs(pitch-c["pose"][0])>POSE_DEG or abs(yaw-c["pose"][1])>POSE_DEG:
                    c["pose"],c["pose_t"]=(pitch,yaw),now

                c["flow_ok"]=flow_ok

                live = ((now-c["move_t"]<WIN_SEC) and
                        (c["blinks"]>=BLINK_REQ or now-c["pose_t"]<WIN_SEC) and
                        c["flow_ok"])

                if match and not live:
                    match=False; name="SPOOF?"

                color=(0,255,0) if match else (0,0,255)
                cv2.rectangle(frame,(l,t),(r,b),color,2)
                cv2.rectangle(frame,(l,b-35),(r,b),color,cv2.FILLED)
                cv2.putText(frame,name,(l+6,b-6),
                            cv2.FONT_HERSHEY_SIMPLEX,0.9,(255,255,255),2)

                if match:
                    if mark(name):           # baru dicatat → tampil banner hijau
                        ok_until=now+SUCCESS_TIME
                    c["blinks"]=0
                else:
                    warn_until=now+ANOMALY_TIME

        if now<warn_until: warn_banner(frame)
        elif now<ok_until: ok_banner(frame)

        cv2.imshow("Attendance",frame)
        prev=frame.copy(); i+=1
        if cv2.waitKey(1)&0xFF==ord('q'): break

    cap.release(); cv2.destroyAllWindows()

if __name__=="__main__":
    main()


C:\Users\ASUS\AppData\Local\Temp\ipykernel_3484\2256999503.py:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(euler[0]), float(euler[1])        # pitch, yaw


[ATTENDANCE] MUHAMMAD IRGIANSYAH
